<a href="https://colab.research.google.com/github/DanielaManate/SentimentAnalysis-TopicModeling/blob/master/SA3_2_Text_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# for tokenizing
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from gensim.models.fasttext import FastText

from sklearn.decomposition import PCA

import pickle

import plotly.express as px

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path = 'gdrive/My Drive/GH_NLP_Course/Data/'

In [3]:
# %env PYTHONHASHSEED=0

# Reading Dataset

In [4]:
# Reading the dataset
reviews = pd.read_csv(path + 'Input/3.input_data_prepped_bow.csv')
reviews.head()

,rest_id,text,rating,char_count,positive,text_prep,text_prep_tokens,word_len_prep,text_prep_lim,text_prep_lim_tokens,word_len_prep_lim
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4,302,1,arrived reservation pm seat -PRON- right time ...,"['arrived', 'reservation', 'pm', 'seat', '-PRO...",27,arrived reservation pm seat right time restura...,"['arrived', 'reservation', 'pm', 'seat', 'righ...",25
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5,111,1,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",10,receive amazing service food cook right waitre...,"['receive', 'amazing', 'service', 'food', 'coo...",9
2,yGMCl0vYigshkXiZFIDTNw,Breakfast was delicious and waitress was aweso...,5,146,1,breakfast delicious waitress awesome steak egg...,"['breakfast', 'delicious', 'waitress', 'awesom...",15,breakfast delicious waitress awesome steak egg...,"['breakfast', 'delicious', 'waitress', 'awesom...",15
3,yGMCl0vYigshkXiZFIDTNw,"We have eaten here twice, for lunch. The first...",4,710,1,eaten twice lunch first time french dip mom fi...,"['eaten', 'twice', 'lunch', 'first', 'time', '...",75,eaten twice lunch first time french dip mom fi...,"['eaten', 'twice', 'lunch', 'first', 'time', '...",74
4,yGMCl0vYigshkXiZFIDTNw,"awesome service, great food. Buffy was an ama...",5,59,1,awesome service great food buffy amazing server,"['awesome', 'service', 'great', 'food', 'buffy...",7,awesome service great food amazing server,"['awesome', 'service', 'great', 'food', 'amazi...",6


In [5]:
# Convert text_prep to string to counteract issues later
reviews['text_prep'] = reviews['text_prep'].astype(str)
reviews['text_prep_lim'] = reviews['text_prep_lim'].astype(str)

# DTM BOW

In [6]:
# dtm_bow = pd.read_csv(path + 'Output/1.dtm_1_bow.csv')
# dtm_bow.shape
# dtm_bow.head()

# DTM TF-IDF
- TF = # of times word appears in doc / total # of words
- IDF = log (total # of docs / # of docs containing the word)
TF-IDF = TF * IDF

In [7]:
# max_features = If not None, build a vocabulary that only consider the top 
# max_features ordered by term frequency across the corpus.
vec = TfidfVectorizer(max_features = 6000)

tf_idf = vec.fit_transform(reviews.text_prep_lim)
review_words = vec.get_feature_names()

dtm_tf_idf = pd.DataFrame(tf_idf.toarray(), 
                          columns=review_words)

dtm_tf_idf.head()

,aaabar,aaron,abacus,ability,able,absolute,absolutely,absurd,abundance,abundant,ac,acai,accept,acceptable,access,accessible,accident,accidentally,accommodate,accommodating,accomodate,accompaniment,accompany,accord,accordingly,accordion,account,accuracy,accurate,accuse,ace,ache,achieve,acidic,acknowledge,acknowledgement,acoustic,acquire,across,act,...,yelp,yelper,yen,yep,yes,yesterday,yet,yike,yo,yogurt,yoke,yolk,york,yorker,young,younger,youngster,youth,yr,yrs,yuck,yum,yumm,yumminess,yummm,yummo,yummy,yup,zach,zack,zero,zesty,zillion,zipps,ziti,zoe,zone,zucchini,zuchinni,zupa
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
dtm_tf_idf['food']

0       0.064668
1       0.168440
2       0.124776
3       0.000000
4       0.250735
          ...   
9360    0.044254
9361    0.000000
9362    0.188899
9363    0.045726
9364    0.133919
Name: food, Length: 9365, dtype: float64

In [9]:
reviews.text_prep_lim.iloc[250]

'third time drop star guess new spot care service wait food joke doubt come back third time charm'

In [10]:
dtm_tf_idf['food'].iloc[250]

0.08007433498536726

In [11]:
dtm_tf_idf.to_csv(path+ 'Output/2.dtm_2_tfidf.csv', index = False)

In [12]:
dtm_tf_idf.shape

(9365, 5999)

# Word Embeddings: FastText

In [13]:
word_tokenized_corpus = reviews.text_prep_lim.apply(word_tokenize).to_list()

In [14]:
# final_size = 60 
# number of dimensions => every word will be represented as a 60-dimensional vector

# window_size = 10
# number of words before and after the word

# min_word = 3
# if freq < min_word, the word will not be represented

In [15]:
%%time

model = FastText(size=60,
            window=10,
            min_count=3)

model.build_vocab(sentences=word_tokenized_corpus)

model.train(sentences=word_tokenized_corpus,
            total_examples = len(word_tokenized_corpus),
            epochs=50)

CPU times: user 9min 15s, sys: 571 ms, total: 9min 15s
Wall time: 4min 45s


In [16]:
# In the output above, you can see a 60-dimensional vector for the word "food"
print(model.wv['food'])

[-2.1522546  -1.4004645  -2.3430803   2.304247    1.8472062  -0.18010801
 -0.4194802   2.0289268  -1.1881473  -2.132256    1.1727856   2.5449147
  1.7523258  -0.09163552 -1.3859296  -1.6479983  -0.18210374  0.6263018
 -1.4589245  -2.9572248  -0.21758443 -0.14563651  1.4301544  -4.7971196
  2.5796359  -0.80668044  1.5288422  -1.041913   -2.0071197  -3.5219088
 -0.48446405 -0.6600899  -0.3886861   2.5544906   2.6786668   0.5431208
  4.979155   -1.2626758   3.2563834  -0.9356235   1.5636026   0.4234253
 -3.8257673  -0.06380803  2.3390236  -2.7339213  -0.31479117  1.9020698
 -1.8298345   0.9898066  -4.9292674   1.4850007   0.86092776 -1.1745561
 -0.58485395  0.14100896  0.47938612 -3.9650917   1.1029458  -0.8804468 ]


In [17]:
model.wv.most_similar('order', topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



[('border', 0.8433787822723389),
 ('ordered', 0.7949652671813965),
 ('orderly', 0.7242372035980225),
 ('ordering', 0.7031099796295166),
 ('mine', 0.6911537647247314)]

In [18]:
model.wv.most_similar('great', topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



[('excellent', 0.8483587503433228),
 ('awesome', 0.7930831909179688),
 ('excellently', 0.7855732440948486),
 ('fantastic', 0.7824568748474121),
 ('greatly', 0.7662798166275024)]

In [19]:
# We can also find the cosine similarity between the vectors for any two words, 
# as shown below:

print(model.wv.similarity(w1='order', w2='wait'))
print(model.wv.similarity(w1='slow', w2='slowly'))
print(model.wv.similarity(w1='slow', w2='fast'))

0.34987175
0.8062905
0.29428115


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



## Save model

In [20]:
pickle.dump(model, open(path + 'Output/3.fast_text.pkl', 'wb' ))

# # Load pickle
# model = pickle.load(open(path + 'Output/3.fast_text.pkl', 'rb' ))

## Visualizing Word Similarities

In [21]:
final_list = ['food', 'place', 'good', 'go', 'order', 'get', 'great', 'time', 
              'come', 'service', 'back', 'like', 'would', 'try', 'one', 'make', 
              'love', 'take', 'say', 'chicken', 'wait', 'restaurant', 'eat', 'really', 
              'pizza', 'well', 'delicious', 'always', 'also', 
              'ask']

In [22]:
X = model.wv[final_list]
X

array([[-2.1522546 , -1.4004645 , -2.3430803 , ..., -3.9650917 ,
         1.1029458 , -0.8804468 ],
       [-0.59658444,  1.1234553 , -1.3232585 , ..., -1.1016197 ,
         0.27277482, -1.1767603 ],
       [ 1.6225928 , -3.4526818 , -1.0109891 , ...,  0.0430108 ,
         2.5686922 ,  1.0881162 ],
       ...,
       [ 0.02272236, -0.08490023, -0.57823586, ..., -0.23411596,
        -0.08179051,  1.0059556 ],
       [-1.3481609 , -2.6623898 ,  2.1988752 , ...,  0.92497057,
         4.1956315 ,  1.5721879 ],
       [-4.1742606 ,  2.0421386 ,  8.334456  , ...,  2.5185506 ,
        -6.649186  ,  3.7773998 ]], dtype=float32)

In [23]:
pca = PCA(n_components=2).fit_transform(X)

In [24]:
fig = px.scatter(x=pca[:, 0], y=pca[:, 1],
                 text = final_list)
fig.update_traces(textposition='top center', 
                  textfont_size=14)
fig.show()